In [ ]:
# Using Tkinter for GUI
# and PyInstaller for executable https://www.youtube.com/watch?v=QWqxRchawZY
# https://medium.com/swlh/easy-steps-to-create-an-executable-in-python-using-pyinstaller-cc48393bcc64

In [4]:
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog, messagebox

from tensorflow import keras
import cv2
import numpy as np

In [54]:
root = Tk()
root.title("FYP GUI")

def openimg():
    global img
    root.filename = filedialog.askopenfilename(title="Select A File", filetypes=(("png files", "*.png"), ("all files", "*.*")))
    image_path_label = Label(root, text=root.filename).pack()
    img = ImageTk.PhotoImage(Image.open(root.filename))
    img_label = Label(image=img).pack()

img_btn = Button(root, text="Open Image", command=openimg)
img_btn.pack()

root.mainloop()

In [9]:
root = Tk()
root.title("FYP GUI")

# Global var
img_dict = {"bound": "", "mask": "", "original": ""} 

# Functions
def open(filetype):
    if filetype == "image":
        root.filename = filedialog.askopenfilename(initialfile=img_url_entry.get()
                                                   ,title="Select A File", 
                                                    filetypes=(("png files", "*.png"), 
                                                                ("jpg files", "*.jpg"),
                                                                ("all files", "*.*")))
        img_url_entry.delete(0, END)
        img_url_entry.insert(0, root.filename)
        preview(root.filename)

    elif filetype == "model":
        root.filename = filedialog.askopenfilename(initialfile=model_url_entry.get()
                                                   ,title="Select A File", 
                                                    filetypes=(("h5 files", "*.h5"),
                                                                ("all files", "*.*")))
        model_url_entry.delete(0, END)
        model_url_entry.insert(0, root.filename)

def preview(url):
    global tkimg
    img = Image.open(url)
    resized_image = img.resize((400, 400), Image.Resampling.LANCZOS)
    tkimg = ImageTk.PhotoImage(resized_image)
    
    img_label = Label(frame_left, image=tkimg)
    img_label.grid(row=6, column=0, columnspan=3)

def detect():
    global img_dict
    try:
        SIZE_X = 256
        SIZE_Y = 256
        connectivity = 4
        img_path = img_url_entry.get()
        model_path = model_url_entry.get()
        model_test = keras.models.load_model(model_path, compile=False)

        # preparing given image
        test_img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_dict["original"] = test_img
        test_img = cv2.resize(test_img, (SIZE_Y,SIZE_X))
        test_img = np.expand_dims(test_img, axis=0)

        # inference to get predicted mask
        pred_mask = model_test.predict(test_img)
        pred_mask = pred_mask.reshape((SIZE_X, SIZE_Y))
        pred_mask = (255 * pred_mask).astype(np.uint8)  # convert values from 0-1 to 0-255
        thresh_val, thresh_pred_img = cv2.threshold(pred_mask, 120, 255, cv2.THRESH_BINARY + 
                                                cv2.THRESH_OTSU)    # thresholding to ensure 0 to 255
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh_pred_img, 
                                                                                connectivity, 
                                                                                cv2.CV_32S) # detect connected blobs
        
        # drawing bounding circles in original image
        img_bounds = test_img.reshape((SIZE_X, SIZE_Y, 3))
        img_bounds = cv2.cvtColor(img_bounds, cv2.COLOR_BGR2RGB)
        color = (0, 255, 0)
        thickness = 1

        for i in range(num_labels-1):
            # cv2.circle(image, center_coordinates, radius, color, thickness)
            center_coor = [int(center) for center in centroids[i+1].tolist()]
            cv2.circle(img_bounds, center_coor, stats[i+1, cv2.CC_STAT_WIDTH], color, thickness)

        # display
        img_dict["bound"] = img_bounds
        img_dict["mask"] = thresh_pred_img
        display(img_dict, "bound")
        countdefects(num_labels-1)
    except:
        print("Exception found in detect(): Model and image file path must be included. Image size should be 512x512")
        messagebox.showerror("Error!", "Exception found in detect(): Model and image file path must be included. Image size should be 512x512")

def display(imgs, type):
    global tkpred
    try:
        if type == "bound":
            im_pil = Image.fromarray(imgs[type])
            resized_image = im_pil.resize((512, 512), Image.Resampling.LANCZOS)
            tkpred = ImageTk.PhotoImage(resized_image)
            predimg_label = Label(frame_right_bottom, image=tkpred)
            predimg_label.grid(row=0, column=0)
        elif type == "mask":
            im_pil = Image.fromarray(imgs[type])
            resized_image = im_pil.resize((512, 512), Image.Resampling.LANCZOS)
            tkpred = ImageTk.PhotoImage(resized_image)
            predimg_label = Label(frame_right_bottom, image=tkpred)
            predimg_label.grid(row=0, column=0)
        elif type == "original":
            im_pil = Image.fromarray(cv2.cvtColor(imgs[type], cv2.COLOR_BGR2RGB))
            resized_image = im_pil.resize((512, 512), Image.Resampling.LANCZOS)
            tkpred = ImageTk.PhotoImage(resized_image)
            predimg_label = Label(frame_right_bottom, image=tkpred)
            predimg_label.grid(row=0, column=0)
    except:
        print("Exception found in display()")

def countdefects(num_defects):
    defect_density = num_defects / (0.0005*0.0005)
    defect_lbl.config(text = f"Defect detected: {num_defects} |    Defect density: {defect_density:.2e} per cm^2")

# Defining content
frame_left = LabelFrame(root, width=450, height=605)
frame_right = LabelFrame(root, width=550, height=605, padx=5, pady=5, bd=0)
frame_right_top = LabelFrame(frame_right, width=520, bd=0)
frame_right_bottom = LabelFrame(frame_right, width=520, height=520)

img_text_lbl = Label(frame_left, text="Image File:", width=50, anchor="w")
img_url_entry = Entry(frame_left, bg="light yellow", width=50)
img_btn = Button(frame_left, text="Open Image", command=lambda: open("image"))

model_text_lbl = Label(frame_left, text="Model File:", width=50, anchor="w")
model_url_entry = Entry(frame_left, bg="light yellow", width=50)
model_btn = Button(frame_left, text="Open Model", command=lambda: open("model"))

start_btn = Button(frame_left, text="Start", command=detect, width=8)
preview_lbl = Label(frame_left, text="Preview:", width=50, anchor="w")

bound_btn = Button(frame_right_top, text="Detected", command=lambda: display(img_dict, "bound"))
mask_btn = Button(frame_right_top, text="Mask", command=lambda: display(img_dict, "mask"))
original_btn = Button(frame_right_top, text="Original", command=lambda: display(img_dict, "original"))
defect_lbl = Label(frame_right, text="Defect detected:  |    Defect density:   per cm^2", fg="red")

# Positioning
frame_left.grid(row=0, column=0, padx=5, pady=10)
frame_right.grid(row=0, column=1, padx=5, pady=10)
frame_right_top.grid(row=0, column=0)
frame_right_bottom.grid(row=1, column=0)
frame_left.grid_propagate(False)
frame_right.grid_propagate(False)
frame_right_bottom.grid_propagate(False)

## left frame
img_text_lbl.grid(row=0, column=0, columnspan=2)
img_url_entry.grid(row=1, column=0, columnspan=2)
img_btn.grid(row=1, column=2, padx=4)

model_text_lbl.grid(row=2, column=0, columnspan=2)
model_url_entry.grid(row=3, column=0, columnspan=2)
model_btn.grid(row=3, column=2, padx=4)

start_btn.grid(row=4, column=2, pady=10)
preview_lbl.grid(row=5, column=0, columnspan=2)

## right frame
bound_btn.grid(row=0, column=0)
mask_btn.grid(row=0, column=1)
original_btn.grid(row=0, column=2)
defect_lbl.grid(row=2, column=0)

root.mainloop()